<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [4]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [5]:
import folium
import pandas as pd

In [6]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [10]:
# If you have the dataset locally saved
spacex_df = pd.read_csv("dataset_part_2.csv")

# Or use your previously processed dataset
spacex_df = pd.read_csv("dataset_part_3.csv")


<class 'FileNotFoundError'>: [Errno 44] No such file or directory: 'dataset_part_2.csv'

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [12]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [13]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [15]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [21]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [22]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [23]:
# Task 2: Mark the success/failed launches for each site on the map
# Use the original spacex_df with 'class' column (0=failure, 1=success)
print("Checking class distribution:")
print(spacex_df['class'].value_counts())

# Check sample data
print("\nSample data:")
print(spacex_df[['Launch Site', 'Lat', 'Long', 'class']].head())


Checking class distribution:
class
0    32
1    24
Name: count, dtype: int64

Sample data:
   Launch Site        Lat       Long  class
0  CCAFS LC-40  28.562302 -80.577356      0
1  CCAFS LC-40  28.562302 -80.577356      0
2  CCAFS LC-40  28.562302 -80.577356      0
3  CCAFS LC-40  28.562302 -80.577356      0
4  CCAFS LC-40  28.562302 -80.577356      0


In [24]:
# Create a new map for success/failure visualization
marker_map = folium.Map(
    location=[29.559684888503615, -95.0830971930759],  # NASA JSC reference
    zoom_start=5  # Zoom out to see all launch sites
)

# Add NASA JSC reference marker
nasa_coordinate = [29.559684888503615, -95.0830971930759]
folium.Marker(
    nasa_coordinate,
    popup="<b>NASA Johnson Space Center</b><br>Mission Control",
    tooltip="NASA JSC",
    icon=folium.Icon(color='blue', icon='star', prefix='fa')
).add_to(marker_map)


In [25]:
# Iterate through each launch record
for index, launch in spacex_df.iterrows():
    # Determine marker color based on success (class)
    if launch['class'] == 1:
        marker_color = 'green'  # Success
        marker_icon = 'ok-sign'
    else:
        marker_color = 'red'    # Failure  
        marker_icon = 'remove-sign'
    
    # Create marker for this launch
    folium.Marker(
        location=[launch['Lat'], launch['Long']],
        popup=f"""
        <b>{launch['Launch Site']}</b><br>
        <b>Outcome:</b> {'Success' if launch['class'] == 1 else 'Failure'}<br>
        Coordinates: {launch['Lat']:.4f}, {launch['Long']:.4f}
        """,
        tooltip=f"{launch['Launch Site']}: {'Success' if launch['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=marker_color, icon=marker_icon, prefix='glyphicon')
    ).add_to(marker_map)

# Display the map
marker_map


In [26]:
# Alternative: Use MarkerCluster for better visualization when many markers overlap
from folium.plugins import MarkerCluster

# Create map with clustering
cluster_map = folium.Map(
    location=[31.0, -85.0],  # Center between sites
    zoom_start=4
)

# Create separate clusters for success and failure
success_cluster = MarkerCluster(name="Successful Launches").add_to(cluster_map)
failure_cluster = MarkerCluster(name="Failed Launches").add_to(cluster_map)

# Add markers to appropriate clusters
for index, launch in spacex_df.iterrows():
    marker = folium.Marker(
        location=[launch['Lat'], launch['Long']],
        popup=f"""
        <b>{launch['Launch Site']}</b><br>
        <b>Outcome:</b> {'Success' if launch['class'] == 1 else 'Failure'}
        """,
        tooltip=f"{launch['Launch Site']}: {'Success' if launch['class'] == 1 else 'Failure'}",
        icon=folium.Icon(
            color='green' if launch['class'] == 1 else 'red',
            icon='ok-sign' if launch['class'] == 1 else 'remove-sign',
            prefix='glyphicon'
        )
    )
    
    # Add to appropriate cluster
    if launch['class'] == 1:
        marker.add_to(success_cluster)
    else:
        marker.add_to(failure_cluster)

# Add layer control
folium.LayerControl().add_to(cluster_map)

cluster_map


In [27]:
# Instead of just calling site_map, try:
from IPython.display import display
display(site_map)

# Or save and display as HTML
site_map.save('temp_map.html')
from IPython.display import HTML
HTML('<iframe src="temp_map.html" width="100%" height="500"></iframe>')


/lib/python3.12/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [28]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [31]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [33]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

In [35]:
# TODO: Create a new column in spacex_df dataframe called marker_color to store the marker colors based on the class value

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Verify the new column
print("New marker_color column created:")
print(spacex_df[['Launch Site', 'class', 'marker_color']].head(10))


New marker_color column created:
   Launch Site  class marker_color
0  CCAFS LC-40      0          red
1  CCAFS LC-40      0          red
2  CCAFS LC-40      0          red
3  CCAFS LC-40      0          red
4  CCAFS LC-40      0          red
5  CCAFS LC-40      0          red
6  CCAFS LC-40      0          red
7  CCAFS LC-40      0          red
8  CCAFS LC-40      0          red
9  CCAFS LC-40      0          red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [36]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [37]:
# TODO: For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster

# Iterate through each row in spacex_df
for index, record in spacex_df.iterrows():
    # Create folium marker for each launch
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f"""
        <b>Launch Site:</b> {record['Launch Site']}<br>
        <b>Launch Outcome:</b> {'Success' if record['class'] == 1 else 'Failure'}<br>
        <b>Class:</b> {record['class']}
        """,
        tooltip=f"{record['Launch Site']}: {'Success' if record['class'] == 1 else 'Failure'}",
        icon=folium.Icon(
            color=record['marker_color'],  # Use the new marker_color column
            icon='ok-sign' if record['class'] == 1 else 'remove-sign',
            prefix='glyphicon'
        )
    )
    
    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

print(f"Added {len(spacex_df)} markers to the cluster!")


Added 56 markers to the cluster!


In [38]:
# Create the map
sites_map = folium.Map(location=[29.0, -83.0], zoom_start=6)

# Add the marker cluster to the map
sites_map.add_child(marker_cluster)

# Display the map
sites_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [43]:
from IPython.display import display

# Instead of just calling site_map, try:
display(site_map)

# Or for marker cluster map:
display(sites_map)


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [44]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [40]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [45]:
from IPython.display import display

# Instead of just calling site_map, try:
display(site_map)

# Or for marker cluster map:
display(sites_map)


In [46]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [47]:
# Launch site coordinates (CCAFS SLC-40)
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Find coordinate of the closest coastline using MousePosition tool
# Example coordinates (you should get these from map exploration):
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance to coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance to closest coastline: {distance_coastline:.2f} km")


Distance to closest coastline: 0.51 km


In [48]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )

# Add the distance marker to the map
site_map.add_child(distance_marker)


In [49]:
# Instead of just calling site_map, try:
from IPython.display import display
display(site_map)

# Or save and display as HTML
site_map.save('temp_map.html')
from IPython.display import HTML
HTML('<iframe src="temp_map.html" width="100%" height="500"></iframe>')


/lib/python3.12/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

In [50]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )

# Add the distance marker to the map
site_map.add_child(distance_marker)


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [51]:
# TODO: Draw a PolyLine between a launch site to the selected coastline point
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = [
    [launch_site_lat, launch_site_lon],  # Launch site
    [coastline_lat, coastline_lon]       # Closest coastline point
]

lines = folium.PolyLine(
    locations=coordinates, 
    weight=3,
    color='blue',
    opacity=0.8,
    popup=f"Distance to Coastline: {distance_coastline:.2f} km"
)

# Add the line to the map
site_map.add_child(lines)

# Display updated map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [52]:
from math import sin, cos, sqrt, atan2, radians
from folium.features import DivIcon
import folium

# Launch site coordinates (CCAFS SLC-40)
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Coastline coordinates (find using MousePosition tool)
coastline_lat = 28.56367   # Example - replace with your findings
coastline_lon = -80.57163  # Example - replace with your findings

# Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Add launch site marker
launch_marker = folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="<b>CCAFS SLC-40</b><br>Launch Site",
    icon=folium.Icon(color='red', icon='rocket', prefix='fa')
)
site_map.add_child(launch_marker)

# Add coastline marker with distance
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    popup=f"<b>Closest Coastline</b><br>Distance: {distance_coastline:.2f} km",
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
site_map.add_child(distance_marker)

# Draw line between launch site and coastline
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(
    locations=coordinates, 
    weight=3,
    color='blue',
    opacity=0.8
)
site_map.add_child(lines)

print(f"Distance from CCAFS SLC-40 to closest coastline: {distance_coastline:.2f} km")
site_map


Distance from CCAFS SLC-40 to closest coastline: 0.51 km


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [53]:
# Launch site coordinates
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Infrastructure coordinates (find these using MousePosition tool)
proximities = {
    'Closest City (Cocoa Beach)': {'lat': 28.3200, 'lon': -80.6077, 'color': 'green'},
    'Highway US-1': {'lat': 28.5650, 'lon': -80.5900, 'color': 'purple'},
    'Railway (FEC)': {'lat': 28.5500, 'lon': -80.5800, 'color': 'orange'},
    'Orlando Airport': {'lat': 28.4294, 'lon': -81.3089, 'color': 'blue'},
    'Port Canaveral': {'lat': 28.4072, 'lon': -80.5931, 'color': 'darkred'}
}


In [54]:
# Calculate distances and add markers for each proximity point
for location_name, coords in proximities.items():
    # Calculate distance
    distance = calculate_distance(
        launch_site_lat, launch_site_lon, 
        coords['lat'], coords['lon']
    )
    
    # Create marker with distance label
    distance_marker = folium.Marker(
        [coords['lat'], coords['lon']],
        popup=f"""
        <b>{location_name}</b><br>
        Distance from CCAFS SLC-40: {distance:.2f} km
        """,
        icon=DivIcon(
            icon_size=(50, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 10px; color:{coords["color"]}; background-color:white; border:1px solid {coords["color"]}; padding:2px;"><b>{location_name}<br>{distance:.1f} KM</b></div>'
        )
    )
    
    # Add marker to map
    site_map.add_child(distance_marker)
    
    # Draw line between launch site and infrastructure
    line_coords = [
        [launch_site_lat, launch_site_lon],  # Launch site
        [coords['lat'], coords['lon']]       # Infrastructure point
    ]
    
    polyline = folium.PolyLine(
        locations=line_coords,
        weight=2,
        color=coords['color'],
        opacity=0.7,
        popup=f"{location_name}: {distance:.2f} km"
    )
    
    # Add line to map
    site_map.add_child(polyline)
    
    print(f"{location_name}: {distance:.2f} km")


Closest City (Cocoa Beach): 27.22 km
Highway US-1: 1.30 km
Railway (FEC): 1.50 km
Orlando Airport: 73.09 km
Port Canaveral: 17.42 km


In [55]:
# Add prominent launch site marker
launch_site_marker = folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="<b>CCAFS SLC-40</b><br>SpaceX Launch Site",
    tooltip="CCAFS SLC-40",
    icon=folium.Icon(color='red', icon='rocket', prefix='fa', icon_size=(30, 30))
)
site_map.add_child(launch_site_marker)

# Display the complete proximity analysis map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
